In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
from matplotlib import pyplot as plt
from numpy.polynomial import polynomial

/glade/work/jhayron/conda-envs/cnn_wr/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def fourierfilter(dataarray,cutoff_period=10):
    # Compute the Fourier transform along the time axis
    fft_data = np.fft.fft(dataarray, axis=0)
    # Get the frequencies corresponding to the FFT components
    freqs = np.fft.fftfreq(dataarray.shape[0], d=1)  # d=1 assumes daily data; adjust if different
    
    # Compute the corresponding periods (in days)
    periods = np.abs(1 / freqs)
    
    # Define the cutoff period for high-pass filter (10 days)
    cutoff_period = 10
    
    # Create a mask to filter out low-frequency components (longer than 10 days)
    high_pass_mask = periods < cutoff_period
    
    # Apply the mask to the FFT data (set low-frequency components to zero)
    fft_data_filtered = fft_data.copy()
    fft_data_filtered[high_pass_mask, :, :] = 0
    
    # Perform the inverse FFT to get the filtered data back in the time domain
    filtered_data = np.fft.ifft(fft_data_filtered, axis=0).real
    
    # Create a new xarray DataArray to store the filtered data
    filtered_anomalies = xr.DataArray(
        filtered_data,
        dims=dataarray.dims,
        coords=dataarray.coords,
        attrs=dataarray.attrs
    )
    return filtered_anomalies

def compute_anoms_experiment_complete(name_reanalysis):
    name_experiment = unique_names_experiments[id_experiment]
    print(f'Started {name_experiment}')
    where_files = np.where(names_experiments_all==name_experiment)[0]
    files_temp = filenames[where_files]
    dataset_temp = extractz500_several_files(files_temp)
    
    climatology_temp, climatology_std_temp = compute_climatology(dataset_temp)
    
    anoms = (dataset_temp.Z500 - climatology_temp.Z_climo)/climatology_std_temp.Z_climo
    del(climatology_temp)
    del(climatology_std_temp)
    filtered_anoms = fourierfilter(anoms)
    
    filtered_anoms = filtered_anoms.to_dataset(name='Z_anoms')
    filtered_anoms.to_netcdf(f'{path_outputs_anoms}anoms_{name_experiment}.nc')
    print(f'Experiment {name_experiment} complete')

# ERA5 Example

In [3]:
def detrend_obs(data, train_data, npoly=3):
    '''
    detrend reanalysis using polynomial fit (for each doy) to the training mean
    
    data: [time, lat, lon] or [member, time]
        reanalysis to detrend 
    
    train_data: [time, lat, lon] or [time]
        ensemble mean 
    
    npoly: [int] 
        order of polynomial, default = 3rd order
    '''
    # stack lat and lon of ensemble mean data
    if len(train_data.shape) == 3:
        train_data = train_data.stack(z=('lat', 'lon'))
 
    # stack lat and lon of member data & grab doy information
    if len(data.shape) == 3:
        data = data.stack(z=('lat', 'lon'))
    temp = data['time.dayofyear']
    
    # grab every Xdoy from ensmean, fit npoly polynomial
    # subtract polynomial from every Xdoy from members
    detrend = []
    for label,ens_group in train_data.groupby('time.dayofyear'):
        Xgroup = data.where(temp == label, drop = True)
        
        curve = polynomial.polyfit(np.arange(0, ens_group.shape[0]), ens_group, npoly)
        trend = polynomial.polyval(np.arange(0, ens_group.shape[0]), curve, tensor=True)
        if len(train_data.shape) == 2: #combined lat and lon, so now 2
            trend = np.swapaxes(trend,0,1) #only need to swap if theres a space dimension

        diff = Xgroup - trend
        detrend.append(diff)

    detrend_xr = xr.concat(detrend,dim='time').unstack()
    detrend_xr = detrend_xr.sortby('time')
    
    return detrend_xr

def smooth_standard_deviation(std_doy, window=60):
    # Extend the array by wrapping around for edge effects
    extended_std_doy = xr.concat([std_doy[-window:], std_doy, std_doy[:window]], dim='dayofyear')
    # Apply rolling mean and remove the extra days
    smoothed_std = extended_std_doy.rolling(dayofyear=window, center=True).mean()
    smoothed_std = smoothed_std[window:-window]
    return smoothed_std

def standardize_anomalies_with_smoothed_std(da):
    # Compute day of year
    doy = da['time'].dt.dayofyear
    
    # Group data by day of year and compute standard deviation
    std_doy = da.groupby(doy).std('time')
    
    # Smooth the standard deviation using a 60-day rolling average
    smoothed_std_doy = smooth_standard_deviation(std_doy, window=60)
    # return smoothed_std_doy
    # Standardize the anomalies by dividing by the smoothed standard deviation
    standardized_da = da.groupby(doy) / smoothed_std_doy
    
    return standardized_da

# # Example usage:
# # standardized_anomalies = standardize_anomalies_with_smoothed_std(da)

In [4]:
path_origins = '/glade/derecho/scratch/jhayron/Data4WRsClimateChange/ProcessedDataReanalyses/'
name_reanalysis = 'ERA5'
dataset = xr.open_dataset(f'{path_origins}Z500_{name_reanalysis}.nc')
anoms = detrend_obs(dataset.Z,dataset.Z)
std_anoms = standardize_anomalies_with_smoothed_std(anoms)
filtered_anoms = fourierfilter(std_anoms)
filtered_anoms = filtered_anoms.to_dataset(name='Z_anoms')
filtered_anoms = filtered_anoms.drop_vars('dayofyear')
path_output_anoms = f'{path_origins}Z500Anoms_{name_reanalysis}_v2.nc'
filtered_anoms.to_netcdf(path_output_anoms)

/glade/derecho/scratch/jhayron/tmp/ipykernel_214077/2618834057.py:8: RuntimeWarning: divide by zero encountered in divide
  periods = np.abs(1 / freqs)


# All the others

In [5]:
def compute_anoms_reanalysis(name_reanalysis):
    path_origins = '/glade/derecho/scratch/jhayron/Data4WRsClimateChange/ProcessedDataReanalyses/'
    dataset = xr.open_dataset(f'{path_origins}Z500_{name_reanalysis}.nc')
    anoms = detrend_obs(dataset.Z,dataset.Z)
    std_anoms = standardize_anomalies_with_smoothed_std(anoms)
    filtered_anoms = fourierfilter(std_anoms)
    filtered_anoms = filtered_anoms.to_dataset(name='Z_anoms')
    filtered_anoms = filtered_anoms.drop_vars('dayofyear')
    path_output_anoms = f'{path_origins}Z500Anoms_{name_reanalysis}_v2.nc'
    filtered_anoms.to_netcdf(path_output_anoms)

In [6]:
compute_anoms_reanalysis('JRA3Q')

/glade/derecho/scratch/jhayron/tmp/ipykernel_214077/2618834057.py:8: RuntimeWarning: divide by zero encountered in divide
  periods = np.abs(1 / freqs)


In [7]:
compute_anoms_reanalysis('MERRA2')

/glade/derecho/scratch/jhayron/tmp/ipykernel_214077/2618834057.py:8: RuntimeWarning: divide by zero encountered in divide
  periods = np.abs(1 / freqs)


In [8]:
compute_anoms_reanalysis('NCEP_NCAR')

/glade/derecho/scratch/jhayron/tmp/ipykernel_214077/2618834057.py:8: RuntimeWarning: divide by zero encountered in divide
  periods = np.abs(1 / freqs)


In [9]:
compute_anoms_reanalysis('NCEP_DOE')

/glade/derecho/scratch/jhayron/tmp/ipykernel_214077/2618834057.py:8: RuntimeWarning: divide by zero encountered in divide
  periods = np.abs(1 / freqs)


In [4]:
xr.open_dataset('/glade/derecho/scratch/jhayron/Data4WRsClimateChange/ProcessedDataReanalyses/Z500_ERA5.nc').Z[0]

<xarray.DataArray 'Z' (lat: 81, lon: 360)>
array([[57328.3  , 57342.074, 57356.24 , ..., 57286.574, 57300.156, 57314.832],
       [57324.324, 57337.02 , 57349.72 , ..., 57280.55 , 57294.688, 57309.95 ],
       [57317.875, 57329.676, 57342.176, ..., 57276.98 , 57291.188, 57305.49 ],
       ...,
       [51521.938, 51515.72 , 51509.582, ..., 51540.438, 51534.23 , 51528.082],
       [51686.98 , 51683.47 , 51680.   , ..., 51697.645, 51694.05 , 51690.51 ],
       [51810.05 , 51810.05 , 51810.05 , ..., 51810.05 , 51810.05 , 51810.05 ]],
      dtype=float32)
Coordinates:
    time     datetime64[ns] 1940-01-01
  * lat      (lat) int64 10 11 12 13 14 15 16 17 18 ... 83 84 85 86 87 88 89 90
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359